# Introduction
This notebook demonstrates the integration capabilities of roboquant with Alpaca. Alpaca is a new broker with an "API-first" approach, so a great fit for algoritmic trading. Besides being a broker that offers US stocks & crypto assets for trading, they also provide a nice data api. Under the hood roboquant uses the excellent [alpaca-java](https://petersoj.github.io/alpaca-java/) library by jacob Peterson to do much of the heavy lifting.    

There are currently three types of integration available:

1. Use the historic data from Alpaca
2. Use live data from Alpaca
3. Use the Alpaca trading functionality (WIP)

Alpaca has different subscriptions which offer different functionality. But this notebook should run fine with the free-tier and just the default paper-trading account.  

We'll run a very simple strategy since the focus is Alpaca integration for this notebook. But there are other notebooks available that focus on how to develop custom strategies and other aspect of roboquant like charting. 

In [ ]:
%use roboquant
Welcome()

By default the required Alpaca classes are not imported, so we need to do that first. This will load the Feed and Broker classes for integrating with Alpaca

In [ ]:
import org.roboquant.alpaca.*

# Setup
We now create, as usual, the strategy we want to test. There is nothing specific to Alapca Feed that the strategy needs to be aware of.  And we create an instance of Roboquant with the strategy we just defined and some metrics to later see how we have done. Again this is all independent of the data feed we are going to use. Please note we create a very fast EMA (Exponential Moving Average) Crossover. Not a very common practice, but for the purpose of this demo we don't want to have to wait too long before any results can be observed.  

In [ ]:
val strategy = EMACrossover(3, 5)
val roboquant = Roboquant(strategy, AccountSummary(), ProgressMetric())

# Historic Feed
The first example shows how to use historic data from Alpaca. To use Alpaca you'll need to authenticate with a Key and Secret. There are several ways to pass them to the feed, either directly or through system environment or a property file:

* Setting the environment or JVM variables APCA_API_KEY_ID and APCA_API_SECRET_KEY
* Or pass them as parameters to the *AlpacaFeed* constructor. If you take this approach, make sure you don't accidentally share your notebook with anyone. 
* Or specify them in the dotenv file found in the same directory. The one included in with the notebooks already has the keys defined.



In [ ]:
val feed = AlpacaHistoricFeed()

// We get the data for last 200 days. The minus 15 is to make sure we don't request very recent data 
// that not all subscriptions are entitled to.
val tf = TimeFrame.pastPeriod(200).minus(15)
feed.retrieve("AAPL", "GOOGL", "IBM", timeFrame = tf)
feed.assets.summary()

In [ ]:
roboquant.run(feed)

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
val metric = roboquant.logger.getMetric("account.value")
MetricChart(metric)

# Live Feed
Now we create an instance of the AlpacaFeed and subscribe to one or more assets. We need to provide the API Key and Secret to authenticate against the Alpaca API. We can do this by:

* Setting the environment or JVM variables APCA_API_KEY_ID and APCA_API_SECRET_KEY
* Or pass them as parameters to the *AlpacaFeed* constructor. If you take this approach, make sure you don't accidentally share your notebook with anyone.
* Or set them in the dotenv file found in the same directory

Also if you are going to run this notebook yourself, make sure you **use KEY/SECRET for paper trading since roboquant is still very much in Beta**.


Alpaca offers the prices for many stocks. For this notebook we're going to limit that a bit, we just get all the stocks who's symbol name starts with "A" and then subscribe to them. Alpaca has a generous API rate limit, so it is possible for example to subscribe to all S&P 500 assets.

In [ ]:
// val feed = AlpacaFeed("your api key", "you secret")

val feed = AlpacaFeed()
val assets = feed.assets.filter { it.symbol.startsWith("A") }
feed.subscribe(assets)
assets.summary().print(20)

We have all the components assembled that we need to start the test. All that remains, is to start the run and evaluate the strategy against the feed. 
We'll run it for 30 minutes, but you can change this. But remember the EMA Crossover is only going to evaluate after at least three observations.

If the time that is displayed in the progress bar looks off by a few hours, please realize that roboquant uses a timezone independent representation for all internal time processing (same as UTC).

In [ ]:
val timeFrame = TimeFrame.nextMinutes(30)
roboquant.reset()
roboquant.run(feed, timeFrame)
feed.disconnect()

## Results
The run has completed, lets see a few result. Of course if the run was outside trading hours without any price action, there will not be much to see.

In [ ]:
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
val logger = roboquant.logger
logger.summary().print()

In [ ]:
logger.getMetricNames().summary()

## Charts
Also lets plot two charts. See also the visualization notebook for examples of how to use charts to display results

In [ ]:
val accountValue = logger.getMetric("account.value")
MetricChart(accountValue, useTime = false)

In [ ]:
TradeChart(account.trades)

# Paper Trading
The integration with the trading API is still very much **work in progress** and it is recommended to **only use this with a paper trading account**. And just as it was the case in the previous section, most live feeds will only generate data during trading hours. So if you run these code cells outside trading hours, you won't see signals and orders being generated.

In [ ]:
val broker = AlpacaBroker()

In [ ]:
broker.account.summary()

In [ ]:
broker.account.portfolio.summary()

In [ ]:
val strategy = EMACrossover(3, 5)
val roboquant = Roboquant(strategy, AccountSummary(), ProgressMetric(), broker = broker)

The setup of the feed is exactly the same as before. There is no difference in the feed if you use it for live testing with the builtin Simulated Broker or paper- and live-trading. Alos this time we'll run it for 30 minutes.

In [ ]:
val feed = AlpacaFeed()
val assets = feed.assets.filter { it.symbol.startsWith("A") }
feed.subscribe(assets)

In [ ]:
val timeFrame = TimeFrame.nextMinutes(30)
roboquant.run(feed, timeFrame)
feed.disconnect()

In [ ]:
broker.account.portfolio.summary()